In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
tf.__version__

'2.15.0'

#Part 1: Data Preprocessing

##Image Data Augmentation with ImageDataGenerator
**ImageDataGenerator** in Keras is a powerful utility that allows for the preprocessing and real-time augmentation of image data. This tool is essential for enhancing the diversity of the dataset without physically expanding it, thereby helping in the development of robust machine learning models. Below, we describe the transformations applied to both training and testing datasets:



1.   Training Data Transformations


- Rescaling (Normalization):

 * Purpose: Converts pixel values from a range of [0, 255] to [0, 1], ensuring that the neural network processes inputs that are normalized, leading to more stable and faster convergence.
  * Computer Vision Concept: This is a basic form of data scaling in image processing, critical for balancing input feature scales.

- Shear Transformation:

  * Purpose: Randomly distorts the image along an axis, typically simulating a tilt, which helps the model learn to recognize objects in images that are not perfectly aligned with the axis.
  * Computer Vision Concept: Shear is a type of affine transformation that slants the shape of an image, preserving lines but not distances or angles.

- Zoom Transformation:

  * Purpose: Randomly increases or decreases the size of the image, mimicking the effect of the camera moving closer or farther away. This teaches the model to recognize objects across different scales.
  * Computer Vision Concept: Zoom is a scaling transformation that changes the effective resolution of the imaged objects.

- Horizontal Flip:

  * Purpose: Mirrors the image along the vertical axis, effectively doubling the number of different orientations the model sees during training.
  * Computer Vision Concept: This is a reflection transformation, useful for datasets where object orientation is not a factor in classification accuracy.



2.   Testing Data Transformations
- Rescaling (Normalization):
  * Purpose and Concept: Identical to the training data, ensures that the model evaluates new, unseen images under the same conditions as during training. Normalization is crucial for maintaining consistent input feature scales during both training and testing phases.

In [ ]:
train_data = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range = 0.2, horizontal_flip = True)
test_data = ImageDataGenerator(rescale=1./255)

## Create Training Set

In [ ]:
training_set = train_data.flow_from_directory('/content/drive/MyDrive/dataset/preprocessedTrainingData',
                                                 target_size = (128, 128),
                                                 batch_size = 10,
                                                 color_mode = 'grayscale',
                                                 class_mode = 'categorical')

Found 21605 images belonging to 26 classes.


In [ ]:
testing_set = test_data.flow_from_directory('/content/drive/MyDrive/dataset/preprocessedTestingData',
                                                 target_size = (128, 128),
                                                 batch_size = 10,
                                                 color_mode = 'grayscale',
                                                 class_mode = 'categorical')

Found 5259 images belonging to 26 classes.


# Part 2: Building the CNN

In [ ]:
classifier = tf.keras.models.Sequential()

In [ ]:
classifier.add(tf.keras.layers.Conv2D(filters=32,
                                     kernel_size=3,
                                     padding="same",
                                     activation="relu",
                                     input_shape=[128, 128, 1]))

In [ ]:
classifier.add(tf.keras.layers.MaxPool2D(pool_size=2,
                                         strides=2,
                                         padding='valid'))

In [ ]:
classifier.add(tf.keras.layers.Conv2D(filters=32,
                                      kernel_size=3,
                                      padding="same",
                                      activation="relu"))

classifier.add(tf.keras.layers.MaxPool2D(pool_size=2,
                                         strides=2,
                                         padding='valid'))

In [ ]:
classifier.add(tf.keras.layers.Flatten())

In [ ]:
classifier.add(tf.keras.layers.Dense(units=128,
                                     activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=96, activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=64, activation='relu'))
classifier.add(tf.keras.layers.Dense(units=26, activation='softmax')) # softmax for more than 2

In [ ]:
classifier.compile(optimizer = 'adam',
                   loss = 'categorical_crossentropy',
                   metrics = ['accuracy'])

In [ ]:
classifier.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 128, 128, 32)      320       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 64, 64, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 32, 32, 32)        0         
 g2D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 32768)             0         
                                                                 
 dense_8 (Dense)             (None, 128)              

In [43]:
classifier.fit(training_set,
                  epochs = 5,
                  validation_data = testing_set)

Epoch 1/5
2161/2161 [==============================] - 576s 266ms/step - loss: 0.1040 - accuracy: 0.9674 - val_loss: 9.3331 - val_accuracy: 0.1852
Epoch 2/5
2161/2161 [==============================] - 579s 268ms/step - loss: 0.0868 - accuracy: 0.9729 - val_loss: 9.3226 - val_accuracy: 0.1947
Epoch 3/5
2161/2161 [==============================] - 550s 254ms/step - loss: 0.0706 - accuracy: 0.9785 - val_loss: 9.0114 - val_accuracy: 0.1510
Epoch 4/5
2161/2161 [==============================] - 553s 256ms/step - loss: 0.0652 - accuracy: 0.9797 - val_loss: 10.2375 - val_accuracy: 0.1597
Epoch 5/5
2161/2161 [==============================] - 581s 269ms/step - loss: 0.0621 - accuracy: 0.9817 - val_loss: 12.4722 - val_accuracy: 0.1649


In [44]:
model_json = classifier.to_json()
with open("model_new.json", "w") as json_file:
    json_file.write(model_json)
print('Model Saved')
classifier.save_weights('model_new.h5')
print('Weights saved')

Model Saved
Weights saved
